**IMPORT REQUIRED LIBRARIES**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random

**CONNECTING TO WANDB**

In [2]:
#---------------------------------------install and import wandb -------------------------------------------------
!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 8.1MB/s 
     |████████████████████████████████| 163kB 46.2MB/s 
     |████████████████████████████████| 133kB 35.3MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 


In [3]:
#--------------------------------------------login to wandb --------------------------------------------
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
#--------------------------------caution: terminal commands ---------------------------------------------
%cd
%cd .keras/datasets/
!rm -r *

/root
[Errno 2] No such file or directory: '.keras/datasets/'
/root
rm: cannot remove '*': No such file or directory


In [5]:
########################################### download data from given url ###############################################

dataset_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
data_dir = tf.keras.utils.get_file('dakshina_dataset_v1.0', origin=dataset_url, untar=True)

2008342528/2008340480 [==============================] - 22s 0us/step


In [6]:
#----------------------------------terminal command -----------------------------------------------
%cd /root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons/
!ls

/root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons
hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [7]:
train_data_path = "hi.translit.sampled.train.tsv"
test_data_path = "hi.translit.sampled.test.tsv"
validation_data_path = "hi.translit.sampled.dev.tsv"


**MACHINE TRANSLITERATOR**


In [8]:
class Machine_Transliterator():

  ############################################# constructor for class Machine_Transliterator ##########################################

  def __init__(self,max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, decoder_embed_size,
               num_encoder_layers,num_decoder_layers,epochs, hidden_layer_size,
               num_encoder_tokens, cell_type, num_decoder_tokens,input_token_index, target_token_index, 
               activation="softmax",optimizer="rmsprop",dropout=0.05):
    
    self.cell_type= cell_type
    self.hidden_layer_size = hidden_layer_size  
    self.optimizer = optimizer
    self.activation = activation   
    self.dropout=dropout

    #-------------------------------------- Number of hidden layers -------------------------------------

    self.num_encoder_layers = num_encoder_layers
    self.num_decoder_layers=num_decoder_layers

    #-------------------------------------- sequence length -------------------------------------
    self.max_decoder_seq_length=max_decoder_seq_length
    self.max_encoder_seq_length=max_encoder_seq_length

    #---------------------------------------------Embedding size-------------------------------------
    self.encoder_embed_size = encoder_embed_size
    self.decoder_embed_size = decoder_embed_size
    
    #-----------------information obtained after preprocessing of data-------------------------------------
    self.num_encoder_tokens = num_encoder_tokens
    self.num_decoder_tokens = num_decoder_tokens

    #-----------------------------dictionaries----------------------------------------------------
    self.input_token_index = input_token_index
    self.target_token_index = target_token_index

 


#########################################function to build model ###########################################

  def build_model(self):

    
    # encoder_inputs = keras.Input(shape=(None,))  
    encoder_inputs = keras.Input(shape=(self.max_encoder_seq_length,)) 
    # encoder_embedding_output = tf.keras.layers.Embedding(self.num_encoder_tokens, self.encoder_embed_size)(encoder_inputs)
    encoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_encoder_tokens, output_dim = self.encoder_embed_size)(encoder_inputs)


#------------------------------ if cell type = LSTM -------------------------------------------------------------
    if self.cell_type == "lstm":
     #--------------------- encoder -----------------------------------
      encoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
      encoder_outputs, state_h, state_c = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        encoder_outputs, state_h, state_c = encoder(encoder_outputs)
      encoder_states = [state_h, state_c]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(self.max_decoder_seq_length, ))      
      # decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size)(decoder_inputs)

      decoder = keras.layers.LSTM(self.hidden_layer_size, return_sequences=True, return_state=True,dropout=self.dropout,use_bias=True)
      decoder_outputs, _, _= decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        decoder_outputs, _ , _= decoder(decoder_outputs, initial_state = encoder_states)



#------------------------------ if cell type = Simple RNN -------------------------------------------------------------
    elif self.cell_type == "rnn":
      #--------------------- encoder -----------------------------------
      encoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout, use_bias=True)
      encoder_outputs, state = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout, use_bias=True)
        encoder_outputs, state = encoder(encoder_outputs)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(self.max_decoder_seq_length,))      
      # decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size)(decoder_inputs)

      decoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout, use_bias=True)
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout, use_bias=True)
        decoder_outputs, _= decoder(decoder_outputs, initial_state = encoder_states)



#------------------------------ if cell type = GRU -------------------------------------------------------------
    elif self.cell_type == "gru":
      #--------------------- encoder -----------------------------------
      encoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
      encoder_outputs, state = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        encoder_outputs, state = encoder(encoder_outputs)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(self.max_decoder_seq_length, ))      
      # decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size)(decoder_inputs)
      
      decoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout,use_bias=True)
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        decoder_outputs, _ = decoder(decoder_outputs, initial_state = encoder_states)


    decoder_dense = keras.layers.Dense(self.num_decoder_tokens, activation = self.activation,use_bias=True)
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    #-----------------compile the model -------------------------------------
    model.compile(
         optimizer=self.optimizer,
         loss="categorical_crossentropy",
         metrics=["accuracy"]
         ) 

#-------------------------- return final model ---------------------------------------------------------
    return model





#########################################function for training the model ###########################################

  def train_model(self,encoder_input_data,decoder_input_data,decoder_target_data,epochs,batch_size,
                  val_encoder_input_data, val_decoder_input_data, val_decoder_target_data):
    
     model=self.build_model()
  
     model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size = batch_size,
        epochs = epochs,
        validation_data = ([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
        callbacks=[WandbCallback()]
        )
     return model

#===================================== end of class Machine_Transliterator ==========================================



**PARAMETERS**

In [9]:
cell_type = "gru" # Type of the recurring unit
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
hidden_layer_size= 128  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
activation = "softmax" #activation
# optimizer = "Adam"  #optimizer
encoder_embed_size = 32 #Encoder embedsize
decoder_embed_size = 32 #Decoder embedsize
num_encoder_layers=3  # number of hidden layers in encoder
num_decoder_layers=3   # number of hidden layers in decoder
dropout=0.01#Dropout

In [10]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 1e-2, decay_steps = 140, decay_rate = 0.9)
lr_schedule = (tf.keras.experimental.CosineDecayRestarts(1e-3, 1000))

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

**PREPROCESSING THE DATA**

In [11]:
################################################ preprocessing the train data and getting dictionaries #################################################

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(train_data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text = " " + input_text + " "
    target_text = " " + target_text + " "

    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


Number of samples: 10000
Number of unique input tokens: 27
Number of unique output tokens: 62
Max sequence length for inputs: 20
Max sequence length for outputs: 20


In [12]:
################################################ preparing the data in required format #################################################

def data(path,input_token_index,target_token_index):
  input_texts = []
  target_texts = []
  with open(path, "r", encoding="utf-8") as f:
      lines = f.read().split("\n")
  for line in lines[: min(num_samples, len(lines) - 1)]:
      target_text, input_text, _ = line.split("\t")
      # We use "tab" as the "start sequence" character
      # for the targets, and "\n" as "end sequence" character.
      input_text = " " + input_text + " "
      target_text = " " + target_text + " "

      input_texts.append(input_text)
      target_texts.append(target_text)

  encoder_input_data = np.zeros( (len(input_texts), max_encoder_seq_length), dtype="float32")
  decoder_input_data = np.zeros( (len(input_texts), max_decoder_seq_length), dtype="float32")
  decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t] = input_token_index[char]
      encoder_input_data[i, t + 1 :] = input_token_index[" "]
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t] = target_token_index[char]
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :] = target_token_index[" "]
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0
    
  return (encoder_input_data, decoder_input_data, decoder_target_data)

In [13]:
(encoder_input_data,decoder_input_data,decoder_target_data)=data(train_data_path,input_token_index,target_token_index)
(val_encoder_input_data,val_decoder_input_data,val_decoder_target_data)=data(validation_data_path ,input_token_index,target_token_index)

**CREATING MACHINE TRANSLITERATOR**

In [14]:
# ########################### creating machine transliterator object ###############################
# machine = Machine_Transliterator(
#     max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, 
#     decoder_embed_size,num_encoder_layers,num_decoder_layers,
#     batch_size, hidden_layer_size, num_encoder_tokens, cell_type, num_decoder_tokens, 
#      input_token_index,target_token_index, activation, optimizer,dropout
#     )

In [15]:
# print(encoder_input_data.shape)
# print(decoder_input_data.shape)
# print(decoder_target_data.shape)
# print(v_encoder_input_data.shape)
# print(v_decoder_input_data.shape)
# print(v_decoder_target_data.shape)

**TRAINING**



In [16]:
# model = machine.train_model(
#     encoder_input_data, decoder_input_data,decoder_target_data,epochs,batch_size,
#     v_encoder_input_data, v_decoder_input_data, v_decoder_target_data
#     )

In [17]:
# tf.keras.utils.plot_model(model,to_file='model.png',show_shapes=True)

In [18]:
cell_type = ["lstm","rnn","gru"]
batch_size = [16,32,64,128]  # Batch size for training.
epochs = [10,20,30,50]  # Number of epochs to train for.
hidden_layer_size= [128,256,512]  # Latent dimensionality of the encoding space.
activation = "softmax"
optimizer = "Adam"
num_hidden_layers=[2,5,10]
encoder_embed_size = [27,64] #Encoder embedsize
decoder_embed_size = [27,64] #Decoder embedsize
num_encoder_layers=[1,2,3,4]  # number of hidden layers in encoder
num_decoder_layers=[1,2,3,4]    # number of hidden layers in decoder
dropout=[0.00,0.01, 0.5,0.0001] #Dropout

In [19]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 1e-2, decay_steps = 140, decay_rate = 0.9)
lr_schedule = (tf.keras.experimental.CosineDecayRestarts(1e-3, 1000))

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

In [20]:
sweep_config={
              "method":"random",
              'metric' : {
                            'name' : 'val_accuracy',
                            'goal' : 'maximize',
                         },
          "parameters" : {
                            "cell_type":{"values":["lstm","rnn", "gru"]},
                            "batch_size":{"values": [32,64,128]},
                            "epochs":{"values":[30,40,60]}, 
                            "hidden_layer_size":{"values": [128,256,512]}, 
                            "num_hidden_layers": {"values": [2,5,10]},
                            "encoder_embed_size": {"values": [27,64] },
                            "decoder_embed_size": {"values":[27,64]  },
                            "num_encoder_layers": {"values": [1,2,3,4]},
                            "num_decoder_layers":{"values":[1,2,3,4] },
                            "dropout": {"values":[0.00,0.01, 0.5,0.0001]}
                         }
              }
sweep_id = wandb.sweep(sweep_config, project="MachineTransliterator")

Create sweep with ID: 4qzxmg7y
Sweep URL: https://wandb.ai/anshikag_2210/MachineTransliterator/sweeps/4qzxmg7y


In [21]:
def run():

  wb = wandb.init()
  config = wb.config

  #----------------------sweep parameters------------------------------------
  cell_type =config.cell_type 
  batch_size = config.batch_size 
  epochs = config.epochs 
  hidden_layer_size= config.hidden_layer_size
  encoder_embed_size =config.encoder_embed_size
  decoder_embed_size = config.decoder_embed_size
  num_encoder_layers=config.num_encoder_layers
  num_decoder_layers= config.num_decoder_layers 
  dropout=config.dropout
  
  ########################### creating machine transliterator object ###############################
  machine = Machine_Transliterator(
      max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, 
      decoder_embed_size,num_encoder_layers,num_decoder_layers,
      batch_size, hidden_layer_size, num_encoder_tokens, cell_type, num_decoder_tokens, 
      input_token_index,target_token_index, activation, optimizer,dropout
      )
  model = machine.train_model(
      encoder_input_data, decoder_input_data,decoder_target_data,epochs,batch_size,
      val_encoder_input_data, val_decoder_input_data, val_decoder_target_data
      )  
  return

In [ ]:
wandb.agent(sweep_id, run)

wandb: Agent Starting Run: u9pvdjby with config:
wandb: 	batch_size: 128
wandb: 	cell_type: lstm
wandb: 	decoder_embed_size: 64
wandb: 	dropout: 0.0001
wandb: 	encoder_embed_size: 64
wandb: 	epochs: 60
wandb: 	hidden_layer_size: 512
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 2
wandb: 	num_hidden_layers: 5
wandb: Currently logged in as: anshikag_2210 (use `wandb login --relogin` to force relogin)


Epoch 1/60
79/79 [==============================] - 44s 84ms/step - loss: 1.7562 - accuracy: 0.6578 - val_loss: 1.2352 - val_accuracy: 0.7198
Epoch 2/60
79/79 [==============================] - 5s 60ms/step - loss: 1.0883 - accuracy: 0.7196 - val_loss: 1.2451 - val_accuracy: 0.7234
Epoch 3/60
79/79 [==============================] - 5s 59ms/step - loss: 1.0125 - accuracy: 0.7359 - val_loss: 1.2407 - val_accuracy: 0.7269
Epoch 4/60
79/79 [==============================] - 5s 60ms/step - loss: 0.9263 - accuracy: 0.7514 - val_loss: 1.2350 - val_accuracy: 0.7391
Epoch 5/60
79/79 [==============================] - 5s 61ms/step - loss: 0.8265 - accuracy: 0.7687 - val_loss: 1.1650 - val_accuracy: 0.7456
Epoch 6/60
79/79 [==============================] - 5s 61ms/step - loss: 0.7848 - accuracy: 0.7773 - val_loss: 1.1928 - val_accuracy: 0.7456
Epoch 7/60
79/79 [==============================] - 5s 61ms/step - loss: 0.7324 - accuracy: 0.7889 - val_loss: 1.1994 - val_accuracy: 0.7440
Epoch 8/60
7

epoch,59
loss,0.01208
accuracy,0.99497
val_loss,1.41706
val_accuracy,0.79066
_runtime,351
_timestamp,1619631353
_step,59
best_val_loss,1.07611
best_epoch,16


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▆▅▅▄▄▄▄▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▂▃▃▃▄▄▄▄▅▅▆▆▇▇██████████▇▇████████████
val_loss,▄▄▄▃▃▃▂▃▄▂▃▁▁▁▂▂▂▃▄▃▄▄▄▄▄▄▄▅▅▅▆▆▇▇█▇▇███
val_accuracy,▁▁▃▄▃▄▅▅▃▅▄▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: 1r8yy4y1 with config:
wandb: 	batch_size: 128
wandb: 	cell_type: lstm
wandb: 	decoder_embed_size: 27
wandb: 	dropout: 0.0001
wandb: 	encoder_embed_size: 27
wandb: 	epochs: 60
wandb: 	hidden_layer_size: 256
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 3
wandb: 	num_hidden_layers: 10


...

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▇▆▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇▇██████████████
val_loss,▅▅▅▅▄▄▃▂▁▁▁▂▂▂▃▃▄▄▄▅▅▅▅▆▆▇▇███
val_accuracy,▁▂▃▃▃▃▅▅▆▆▇▇▇▇▇▇▇█████████████
_runtime,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


wandb: Agent Starting Run: mp2bghbs with config:
wandb: 	batch_size: 64
wandb: 	cell_type: gru
wandb: 	decoder_embed_size: 64
wandb: 	dropout: 0
wandb: 	encoder_embed_size: 27
wandb: 	epochs: 60
wandb: 	hidden_layer_size: 128
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 2
wandb: 	num_hidden_layers: 5


Epoch 1/60
157/157 [==============================] - 13s 31ms/step - loss: 1.5081 - accuracy: 0.6755 - val_loss: 1.1977 - val_accuracy: 0.7292


wandb: ERROR Can't save model, h5py returned error: Unable to create link (name already exists)


Epoch 2/60
157/157 [==============================] - 3s 20ms/step - loss: 0.9414 - accuracy: 0.7358 - val_loss: 1.1288 - val_accuracy: 0.7444
Epoch 3/60
157/157 [==============================] - 3s 22ms/step - loss: 0.8548 - accuracy: 0.7584 - val_loss: 1.1166 - val_accuracy: 0.7522
Epoch 4/60
 91/157 [================>.............] - ETA: 1s - loss: 0.8021 - accuracy: 0.7730